In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import scipy.io as sio
from scipy import stats
from scipy import sparse 
from importlib import reload
from matplotlib import pyplot as plt
import sys
#sys.path.append('/home/jma819/post_cmfe_analysis')
sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import python_utils_jjm as utils_jjm
import dlc_utils
import caiman
import statsmodels.formula.api as smf
import scipy.spatial.distance as dist
import itertools
import math
import warnings
import numbers
from multiprocessing import Pool
warnings.filterwarnings(action='once')

In [ ]:
reload(utils_jjm)
reload(dlc_utils)

In [ ]:
cnmfe_file_key = pd.read_csv('/volumes/My_Passport/dlc_analysis/behavcamvideos/cnmfe_file_key.csv')
#cnmfe_file_key = pd.read_csv('/projects/p30771/dlc_analysis/openfield_dlc_output/cnmfe_file_key.csv')

In [ ]:
cnmfe_base_dir = '/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/'
#cnmfe_base_dir = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/'
wt_CNMFE_file = ['30-Mar_20_39_05_out.mat', '30-Mar_20_45_16_out.mat', '27-Feb_17_33_59_out.mat', '22-Mar_22_52_02_out.mat',
                 '28-Feb_16_10_05_out.mat', '27-Feb_17_32_15_out.mat', '28-Feb_16_21_21_out.mat', '25-Mar_13_27_27_out.mat',
                 '25-Mar_14_22_02_out.mat', '25-Mar_14_22_44_out.mat', '26-Mar_18_33_55_out.mat', '27-Mar_00_26_12_out.mat', '27-Mar_00_48_46_out.mat']

ko_CNMFE_files = ['31-Mar_13_28_15_out.mat', '29-Mar_21_42_20_out.mat', '13-Apr_17_57_40_out.mat', '29-Mar_14_27_55_out.mat', '13-Apr_16_01_20_out.mat',
                 '13-Apr_16_11_27_out.mat', '29-Mar_13_39_44_out.mat']

In [ ]:
tracking_files = ['/volumes/My_Passport/dlc_analysis/behavcamvideos/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in wt_CNMFE_file] 
#tracking_files = ['/projects/p30771/dlc_analysis/openfield_dlc_output/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in wt_CNMFE_file] 

In [ ]:
CNMFE_dir_paths_list = [str(cnmfe_base_dir+fname) for fname in wt_CNMFE_file]
#CNMFE_dir_paths_list

In [ ]:
binning_time = 1 # seconds 
body_part_for_tracking = 'tail_base' #
number_of_bins = 50 #
polynomial_degree = 2 #

grouped_raw_data = {}
success = []
failed = []
for CNMFE_file, tracking_file in zip(CNMFE_dir_paths_list, tracking_files):
    #print(CNMFE_file)
    #print(tracking_file)
    try:
        # load cell fluorescence 
        cell_fluorescence = sio.loadmat(CNMFE_file)
        C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
        C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)
        # create tracking time deltas
        interpolated = utils_jjm.prepare_timedelta_dfs(tracking_file)
        #load spatial components by session
        com_df, spatial_components = utils_jjm.return_spatial_info(CNMFE_file, 0.6)
        cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components)
        #C_z_scored_filtered = utils_jjm.filter_out_by_size(C_z_scored, cell_contours, for_dims, 0.6, 100)
        #store results 
        grouped_raw_data[tracking_file.split('/')[-2]] = {'C': C_timedelta, 'C_z_scored': C_z_scored, 'C_normalized': C_normalized, 'C_normalized_z_scored': C_normalized_z_scored, 
                                                          'interpolated' : interpolated, 'com' : com_df, 'spatial_components' : spatial_components, 'cell_contours': cell_contours,  
                                                         'for_dims' : for_dims}
        success.append((tracking_file.split('/')[-2], CNMFE_file.split('/')[-1]))
    except FileNotFoundError:
        failed.append(tracking_file)
    except OSError:
        failed.append(tracking_file)

## spatial clustering

In [ ]:
## combine dfs for each session to bin velocity values across dfs
#downsample
new_sampling_interval = .2
V_df = pd.concat([grouped_raw_data[session]['interpolated'].resample(str(new_sampling_interval)+'S').max() 
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))
all_sessions_v_bins = pd.cut(V_df['tail_base'], bins=50)

V_df['velocity_bins'] = all_sessions_v_bins

In [ ]:
# compile and filter fluorescence
filtered_for_analysis = {}
for session in list(grouped_raw_data.keys()):
    filtered_for_analysis[session] = utils_jjm.filter_out_by_size(grouped_raw_data[session]['C_normalized_z_scored'], grouped_raw_data[session]['cell_contours'], 
                                                                  grouped_raw_data[session]['for_dims'], 0.6, 100)
C_df = pd.concat([filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))


# save data as sparse matricies 

In [ ]:
#session='GRIN013_H13_M33_S54'

coactivity_by_session = {}

for session in tqdm(list(grouped_raw_data.keys())):
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
    com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

    ##get binned fluorescence and calc Jaccard scores
    #arguments are sample widths to bin and z score thresholds
    cells_in_session = C_df.loc[session].dropna(axis=1).drop('msCamFrame', axis=1)
    binned_fluorescence = cells_in_session.apply(utils_jjm.binning_function_uncrop, args=[1, 2])

    reindexed = binned_fluorescence.set_index(int(x) for x in np.linspace(0, len(binned_fluorescence)-1, len(binned_fluorescence)))

#cell_keys_matrix = sparse.dok_matrix((1, len(reindexed.columns)*len(reindexed.columns)), dtype=np.float32)
    cell_pairs = np.array([pair for pair in itertools.combinations(list(reindexed.columns), 2)])
    coactivity_in_session = sparse.dok_matrix((len(reindexed), len(cell_pairs)))
    for time_index in tqdm(range(len(reindexed))):
        #coactivity_by_time_point = sparse.dok_matrix((1, len(cell_pairs)), dtype=np.float32)
        for pair, pair_idx in zip(cell_pairs, range(len(cell_pairs))):
            if (reindexed.loc[time_index][pair[0]] == 1) and (reindexed.loc[time_index][pair[1]] == 1):
                coactivity_in_session[time_index, pair_idx] = 1
            else:
                pass 
    
    coactivity_by_session[session]= {'coactivity_in_session': coactivity_in_session, 'cell_pairs':cell_pairs}
        
    



In [ ]:
#save data to npz format 
for session in tqdm(list(grouped_raw_data.keys())):
    csr_matrix = coactivity_by_session[session]['coactivity_in_session'].tocsc()
    sparse.save_npz("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/"+str(session)+".npz",
                      csr_matrix)
    np.savetxt("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/"+str(session)+".txt", 
               coactivity_by_session[session]['cell_pairs'])
    f.close()

# load saved data and plot spatial coordination index 

In [ ]:
spatial_coordination_by_session = {}
for session in list(grouped_raw_data.keys()):
    print(session)
    session_coactivity = sparse.load_npz("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/"+str(session)+".npz")
    cell_pairs = np.loadtxt("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/"+str(session)+".txt")
# get indicies of small cells 
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                          len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
#compile and filter cell centers of mass
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
    com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

# get one sided ks tests from h5 data set
    binnums = 10
    ks_one_sided_more = []
    ks_one_sided_less = []
    for time_point in tqdm(range(np.shape(session_coactivity)[0])):
        active_cells = [active_cell_pair_idx[1] for active_cell_pair_idx in np.argwhere(session_coactivity[time_point])]
        cell_ids = [sorted(cell_pairs[active_cell_idx]) for active_cell_idx in active_cells]
        active_cell_distances = [com_distances[cell_id[0]][cell_id[1]].values[0] for cell_id in cell_ids]
        cum_results_coactive = stats.cumfreq(active_cell_distances, numbins=binnums, defaultreallimits=(0, 500))
        less_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='less')
        more_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='greater')
        ks_one_sided_more.append(more_result)
        ks_one_sided_less.append(less_result)
    
    spatial_coordination_by_session[session] = {'ks_one_sided_more' : ks_one_sided_more, 'ks_one_sided_less' : ks_one_sided_less}

In [ ]:
## compile coordination index info by session

triggered_activity_across_sessions = {}

for session in list(grouped_raw_data.keys()):
    ks_one_sided_less = spatial_coordination_by_session[session]['ks_one_sided_less']
    coord_index = np.nan_to_num([math.log(result.pvalue, 10)*-1 for result in ks_one_sided_less])

    #put coordination index on time delta dataframe
    coord_index_df = pd.DataFrame(coord_index, columns=['spatial_coordination_index'])
    coord_index_df_time = coord_index_df.set_index(pd.to_timedelta(np.linspace(0, (len(coord_index_df)-1)*(1/5), len(coord_index_df)), unit='s'), drop=True)

    # bin by activity threshold (body_part, resting_time_threshold, active_time_threshold, crossing_threshold, resting_threshold, activity_threshold)
    binned_velocity_df = pd.DataFrame([dlc_utils.bin_by_activity_threshold(V_df.loc[session]['tail_base'], 80, 20, 1, 2, 2) for session in list(grouped_raw_data.keys())], index=list(grouped_raw_data.keys()))
    binned_velocity_df = binned_velocity_df.transpose()

    crossing_indicies = utils_jjm.select_trigger_regions(binned_velocity_df[session], 0.5, 0.5, 20)
    #inputs time_to_plot
    
    threshold_activity = utils_jjm.average_triggered_regions(V_df.loc[session]['tail_base'].values, crossing_indicies, 80)
    
    coordination_threshold_activity = utils_jjm.average_triggered_regions(coord_index, crossing_indicies, 80)
    
    triggered_activity_across_sessions[session] = pd.concat([threshold_activity, coordination_threshold_activity], axis=1, keys=['velocity', 'spatial_coordination'])

In [ ]:
triggered_activity_across_sessions['GRIN013_H13_M33_S54']['spatial_coordination'].head()

In [ ]:
#triggered_activity_across_sessions

In [ ]:
#concat by mouse 
concacted_by_mouse = {}
for mouse in list(set([session[0:7] for session in list(triggered_activity_across_sessions.keys())])):
    dfs_by_mouse = []
    sessions = []
    for session in list(triggered_activity_across_sessions.keys()):
        if mouse in session:
            if not(triggered_activity_across_sessions[session].empty):
                means = triggered_activity_across_sessions[session].mean(axis=1, level=0)
                dfs_by_mouse.append(means)
                sessions.append(session)
    if len(dfs_by_mouse)>0:
        concacted_by_mouse[mouse] = pd.concat(dfs_by_mouse, axis=1, keys=sessions)
combined_by_mouse = pd.concat(list(concacted_by_mouse.values()), axis=1, keys=list(concacted_by_mouse.keys()))

In [ ]:
combined_by_mouse.head()

In [ ]:
len(combined_by_mouse)

In [ ]:
activity_measure = 'spatial_coordination'
x = np.linspace(-(len(combined_by_mouse)/2)*.2, (len(combined_by_mouse)/2)*.2, len(combined_by_mouse))
mean=combined_by_mouse.mean(axis=1, level=2)['velocity'].values
plt.plot(x, combined_by_mouse.mean(axis=1, level=2)['velocity'].values, color='k')
std_error = (combined_by_mouse.std(axis=1, level=2)['velocity'])/math.sqrt(combined_by_mouse.mean(axis=1, level=0).shape[1])
plt.fill_between(x, mean-std_error, mean+std_error)
ax = plt.gca()
ax.axvline(x=(0), linestyle='--', color='r')
plt.show()

In [ ]:
x = np.linspace(-(len(combined_by_mouse)/2)*.2, (len(combined_by_mouse)/2)*.2, len(combined_by_mouse))
mean = combined_by_mouse.mean(axis=1, level=2)[activity_measure].values
plt.plot(x, mean, color='k')
std_error = (combined_by_mouse.std(axis=1, level=2)[activity_measure])/math.sqrt(combined_by_mouse.mean(axis=1, level=0).shape[1])
plt.fill_between(x, mean-std_error, mean+std_error)
ax = plt.gca()
ax.axvline(x=0, linestyle='--', color='r')
plt.show()

In [ ]:
#parallel implementation
def map_to_sparse_matrix(item_idx_tuple, current_row_num, sparse_matrix, orig_df_comparison):
    if orig_df_comparison.loc[current_row_num][item[0]] == 1 and (orig_df_comparison.loc[current_row_num][item[0]] == 1):
        sparse_matrix[current_row_num, item_idx] = 1
    else:
        pass
    return(sparse_matrix)

def map_to_sparse_matrix_time(time_index):
    #for time_index in tqdm(range(len(reindexed))):
    #coactivity_by_time_point = sparse.dok_matrix((1, len(cell_pairs)), dtype=np.float32)
    if time_index%100 == 0:
        print(time_index)
    for pair, pair_idx in zip(cell_pairs, range(len(cell_pairs))):
        if (reindexed.loc[time_index][pair[0]] == 1) and (reindexed.loc[time_index][pair[1]] == 1):
            coactivity_in_session_p[time_index, pair_idx] = 1
        else:
            pass
    return(coactivity_in_session)
    

In [ ]:
# analyze by session in parallel

coactivity_by_session = {}
for session in tqdm(list(grouped_raw_data.keys())):
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
    com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

    ##get binned fluorescence and calc Jaccard scores
    #arguments are sample widths to bin and z score thresholds
    cells_in_session = C_df.loc[session].dropna(axis=1).drop('msCamFrame', axis=1)
    binned_fluorescence = cells_in_session.apply(utils_jjm.binning_function_uncrop, args=[1, 4])

    reindexed = binned_fluorescence.set_index(int(x) for x in np.linspace(0, len(binned_fluorescence)-1, len(binned_fluorescence)))
    
    cell_pairs = np.array([pair for pair in itertools.combinations(list(reindexed.columns), 2)])
    coactivity_in_session_p = sparse.dok_matrix((len(reindexed), len(cell_pairs)))

    p=Pool(3)

    coactivity_in_session = p.map(map_to_sparse_matrix_time, range(len(reindexed)))
    coactivity_by_session[session] = coactivity_in_session

In [ ]:
coactivity_by_session = {}

for session in tqdm(list(grouped_raw_data.keys())):
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
    com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

    ##get binned fluorescence and calc Jaccard scores
    #arguments are sample widths to bin and z score thresholds
    cells_in_session = C_df.loc[session].dropna(axis=1).drop('msCamFrame', axis=1)
    binned_fluorescence = cells_in_session.apply(utils_jjm.binning_function_uncrop, args=[1, 2])

    reindexed = binned_fluorescence.set_index(int(x) for x in np.linspace(0, len(binned_fluorescence)-1, len(binned_fluorescence)))

#cell_keys_matrix = sparse.dok_matrix((1, len(reindexed.columns)*len(reindexed.columns)), dtype=np.float32)
    cell_pairs = np.array([pair for pair in itertools.combinations(list(reindexed.columns), 2)])
    coactivity_in_session = sparse.dok_matrix((len(reindexed), len(cell_pairs)))
    for time_index in tqdm(range(len(reindexed))):
        #coactivity_by_time_point = sparse.dok_matrix((1, len(cell_pairs)), dtype=np.float32)
        for pair, pair_idx in zip(cell_pairs, range(len(cell_pairs))):
            if (reindexed.loc[time_index][pair[0]] == 1) and (reindexed.loc[time_index][pair[1]] == 1):
                coactivity_in_session[time_index, pair_idx] = 1
            else:
                pass 
    
    coactivity_by_session[session]= {'coactivity_in_session': coactivity_in_session, 'cell_pairs':cell_pairs}

In [ ]:
cell_pairs = np.array([pair for pair in itertools.combinations(list(reindexed.columns), 2)])
coactivity_in_session_p = sparse.dok_matrix((len(reindexed), len(cell_pairs)))

p=Pool(3)

coactivity_in_session = p.map(map_to_sparse_matrix_time, range(len(reindexed)))    